In [1]:
#load the file

In [8]:
library(tidyverse)

In [10]:
exam_data <- read_csv("exams.csv")
exam_data

Rows: 1000 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): gender, race/ethnicity, parental level of education, lunch, test pr...
dbl (3): math score, reading score, writing score

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
male,group A,high school,standard,completed,67,67,63
female,group D,some high school,free/reduced,none,40,59,55
male,group E,some college,free/reduced,none,59,60,50
male,group B,high school,standard,none,77,78,68
male,group E,associate's degree,standard,completed,78,73,68
female,group D,high school,standard,none,63,77,76
female,group A,bachelor's degree,standard,none,62,59,63
male,group E,some college,standard,completed,93,88,84
male,group D,high school,standard,none,63,56,65


In [11]:
colnames(exam_data) <- make.names(colnames(exam_data))
exam_data

gender,race.ethnicity,parental.level.of.education,lunch,test.preparation.course,math.score,reading.score,writing.score
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
male,group A,high school,standard,completed,67,67,63
female,group D,some high school,free/reduced,none,40,59,55
male,group E,some college,free/reduced,none,59,60,50
male,group B,high school,standard,none,77,78,68
male,group E,associate's degree,standard,completed,78,73,68
female,group D,high school,standard,none,63,77,76
female,group A,bachelor's degree,standard,none,62,59,63
male,group E,some college,standard,completed,93,88,84
male,group D,high school,standard,none,63,56,65


In [12]:
exam_data |>
mutate(test.preparation.course = as_factor(test.preparation.course)) |>
select(test.preparation.course, math.score, reading.score, writing.score)

test.preparation.course,math.score,reading.score,writing.score
<fct>,<dbl>,<dbl>,<dbl>
completed,67,67,63
none,40,59,55
none,59,60,50
none,77,78,68
completed,78,73,68
none,63,77,76
none,62,59,63
completed,93,88,84
none,63,56,65


In [32]:
exam_data |>
mutate(test.preparation.course = if_else(test.preparation.course=completed, true))
mutate(test.preparation.course = ifelse(test.preparation.course == "completed", 1))




ERROR: [1m[33mError[39m in [1m[1m`mutate()`:[22m
[1m[22m[33m![39m Problem while computing `test.preparation.course =
  if_else(test.preparation.course = completed, true)`.
[1mCaused by error in [1m[1m`if_else()`:[22m
[33m![39m unused argument (test.preparation.course = completed)
